In [ ]:
import os

In [1]:
from pathlib import Path

# Get the current working directory
current_directory = Path.cwd()
print("Current directory:", current_directory)

# Access the parent directory
parent_directory = current_directory.parent
print("Parent directory:", parent_directory)

Current directory: C:\Users\Mukesh\Desktop\numhack_2024\data_ingestion_notebooks
Parent directory: C:\Users\Mukesh\Desktop\numhack_2024


In [2]:
# os.getcwd()

In [3]:
db_path = parent_directory / "numhack_db"
print("Database path:", db_path)

Database path: C:\Users\Mukesh\Desktop\numhack_2024\numhack_db


In [4]:
db_path = r'C:\Users\Mukesh\Desktop\numhack_2024\chatbot\chatapp\numhack_db\\'

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from uuid import uuid4
import chromadb
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key = os.getenv("API_KEY")
weather_api = os.getenv("WEATHER_API")
client = OpenAI(api_key=api_key)


def data_ingestion(file_path, collection_name):
    # file_path = "fishing.pdf"
    loader = PyPDFLoader(file_path)
    
    docs = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200, add_start_index=True
    )
    all_splits = text_splitter.split_documents(docs)
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key)
    # C:\Users\Mukesh\Desktop\numhack_2024
    # parent_directory = Path.cwd().parent
    db_path =  "numhack_db"
    # print(db_path)
    persistent_client = chromadb.PersistentClient(path=db_path)
    collection = persistent_client.get_or_create_collection(collection_name)
    
    vector_store_from_client = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
    )
    
    uuids = [str(uuid4()) for _ in range(len(all_splits))]
    
    vector_store_from_client.add_documents(documents=all_splits, ids=uuids)
    return "data_ingested to vector_db"

In [8]:
data_ingestion("principlespractices_saiplatform_2021.pdf", "agri_collection")

numhack_db


'data_ingested to vector_db'

In [9]:
data_ingestion("fishing.pdf", "fishing_collection")

numhack_db


'data_ingested to vector_db'

In [11]:
def data_retreival(query, collection_name):
    persistent_client = chromadb.PersistentClient(path="numhack_db/")
    collection = persistent_client.get_or_create_collection(collection_name)
    # collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key)
    vector_store_from_client = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,)
    
    results = vector_store_from_client.similarity_search(query=query,k=2)
    retreived_data = ""
    for res in results:
        retreived_data += res.page_content
    return retreived_data

In [12]:
results = data_retreival("give me some sustainable farming techniques", "agri_collection")

In [13]:
results

'SUSTAINABLE AGRICULTURE  \nPRINCIPLES & PRACTICES\n1\nSustainable Agriculture \nPrinciples & Practices•   Implement the use of efficient farming practices; inputs and outputs management, \nsoil health improvement, livestock solutions, and deforestation prevention,  \namong others. \n•   Implement farming systems that increase on-farm biomass, both above and below \nground, and create a significant greenhouse gas sink.\n•   Reduce the use of non-renewable sources of energy and increase the use of \nrenewable sources of energy, and optimise the use of energy-intensive inputs,  \ne.g. inorganic fertilisers.\n•   Manage all sources of emissions which affect air, noise and odour, such as manure \nstorage, burning, agrochemical applications, dust, exhaust, machinery maintenance \netc. to ensure minimal to no emissions. \n•   Drive energy efficiency to avoid wasting energy and increasing the use of renewable \nenergy and fuels on-farm.\n•   Use innovative techniques to identify new carbon si